# Running an interactive Rosetta Session with Narupa

In this notebook we will go over the most fundamental steps in communicating and visualising Rosetta using Narupa.
For this notebook you will need to have downloaded and installed Rosetta from source. 
For more information regarding this please see:
    https://www.rosettacommons.org/
    
Additionally when compiling Rosetta for this purpose you will need to enable both the "cx11threads" and "zeromq" for the necassary executable to run.
This can be done using the following command:
    $ ./scons.py rosetta_interactive mode=release -j4 bin extras=zeromq,cxx11thread


## Initialising the Rosetta Server

First start by starting a Rosetta server. If you have built Rosetta from source this can be accomplished using the following bash command:
    $ rosetta_interactive.cxx11threadzeromq.linuxgccrelease

Please note: That the extension of the executable will change based on your operating system.

By default rosetta_interactive uses the "localhost" address and 43234 port. You may need to alter these settings for your setup. Especially if you intend to run Rosetta and Narupa on seperate machines.


### This notebook will assume the defaults wherever possible.


## Starting up Narupa

With the Rosetta server started we can now initialse the narupa server.
For convenience, we will use the RosettaApplicationServer which contains all the necassary components for this tutorial.
Specifically, this class contains a RosettaCommandService and a RosettaClient which will be used to communicate with Rosetta.

In [ ]:
from narupa.rosetta.server import RosettaApplicationServer
narupa_server = RosettaApplicationServer.basic_server(
    rosetta_address="localhost", rosetta_port=43234 ) 
# Rosetta addres and port have been included for clarity

The narupa_server operates by sending a set of KEY, DATA messages to the Rosetta server. 
The identity of the key dictates how Rosetta should processe the corresponding data.
The RosettaApplicationServer is initialised with a small set of commands which are most commonly going to be used.
Lets have a look at one of them:


In [ ]:
from narupa.rosetta.command_util import EchoMessage
EchoMessage.__doc__

All RosettaCommands are classes derived from a base RosettaCommand class.
When creating new RosettaCommands you should inherit from this class and only alter the ".\_execute" function
In the case of EchoMessage the function takes a string passed to it and sends it to the RosettaClient object and then waits for a reply from the server.


# Making Requests

At this point we will want to start up a NarupaClient to make some requests of our Rosetta server.

In [ ]:
from narupa.core import NarupaClient
client = NarupaClient.insecure_channel( 
    address=narupa_server.address, port=narupa_server.port )

In [ ]:
client.update_available_commands()

All Rosetta commands are prepended with the ROS_ to differentiate them from standard Narupa calls.
Now lets do something interesting and have Rosetta pack and minimise a structure.
First we need to send a structure to Rosetta so it can be stored server side. From there we can ask Rosetta to operate on our structure and request snapshots of the pose as it is being operated on.

In [ ]:
with open( "./resourses/1ubq.pdb" , "r" ) as r:
    pdb = r.readlines()
pdb = "".join(pdb) # We need to convert our list into a str

client.run_command( "ROS_send_pose", pose_to_store = pdb )

From this we can see that the pdb we have given Rosetta has been stored under the alias: "pose0".

To have Rosetta operate on a structure we need to provide it with a set of instructions.
These are in the form of a RosettaScript, see the online documentation for more details on building these:
    https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/RosettaScripts

For this tutorial we will be using the scripts located in the ./resources folder.

In [ ]:
with open( "./resourses/minimise.xml", "r" ) as r:
    xml = r.readlines()
xml = "".join(xml)

client.run_command( "ROS_send_and_parse_xml", xml=xml)

# At this point Rosetta will have parsed the xml we have provided and is now operating on pose0.
# While this is operating lets retrieve some midpoints to visualise.

from time import sleep

frames = [""] * 40

for ii in range(len(frames)):
    try:
        frame = client.run_command( "ROS_request_pose", pose_name="pose0" )
        frames[ii] = frame["pose_pdb"]
    except:
        pass
    sleep( 0.01 )

# Time to remove empty frames
frames = [ frame for frame in frames if frame != "" ]

Now we will convert all of these midpoint pdbs to FrameData objects so they can be visualised.


In [ ]:
from narupa.rosetta.pdb_util import convert_pdb_string_to_framedata

all_framedata = [""] * len(frames)
for ii, frame in enumerate(frames):
    all_framedata[ii] = convert_pdb_string_to_framedata( frame )


In [ ]:
# To visualise the Rosetta data will we setup an loop to run through all of the frames we collected.
from threading import RLock
from concurrent import futures
from time import sleep

playback_fps = 15

class TrajectoryPlayback:       
    """
    Initialise playback, setting things up.
    """
    # Get a pool of threads (just one) that we can run the play back on
    def __init__(self):
        self.threads = futures.ThreadPoolExecutor(max_workers=1)
        self._run_task = None
        self._cancelled = False
        self._cancel_lock = RLock()
        self.frame_index = 0
        self.frames = all_framedata

    @property 
    def is_running(self):
        # Fancy logic that just checks whether or not we're playing the trajectory in the background
        return self._run_task is not None and not (self._run_task.cancelled() or self._run_task.done())

    def play(self):
        """
        Plays the trajectory in the background.
        """
        # First, we have to cancel any existing playback, and start a new one.
        with self._cancel_lock:
            self.cancel_playback(wait=True)
        self.run_playback()
        
    def step(self):
        """
        Take a single step of the trajectory and stop. 
        """
        # The lock here ensures only one person can cancel at a time. 
        with self._cancel_lock:
            self.cancel_playback(wait=True)
            self._step_one_frame()

    def pause(self):
        """
        Pause the playback, by cancelling any current playback.
        """
        with self._cancel_lock:
            self.cancel_playback(wait=True)

    def run_playback(self, block=False):
        """
        Runs the trajectory playback. If block is False, it will run on a background thread.
        """
        if self.is_running:
            raise RuntimeError("The trajectory is already playing on a thread!")
        if block:
            self._run()
        else:
            self._run_task = self.threads.submit(self._run)
    
    def _run(self):
        while not self._cancelled:
            self._step_one_frame()
            sleep( 1 / playback_fps) # Delay sending frames so we hit the desired FPS
        self._cancelled = False
            
        
    def _step_one_frame(self):
        narupa_server.frame_publisher.send_frame(self.frame_index, self.frames[self.frame_index])
        self.frame_index = (self.frame_index + 1) % len(self.frames)

    def cancel_playback(self, wait=False):
        """
        Cancel trajectory playback, if it's running. If wait is True, this method will wait until the playback stops 
        before returning.
        """
        if self._run_task is None:
            return
        if self._cancelled:
            return
        
        self._cancelled = True
        if wait:
            self._run_task.result()
            self._cancelled = False

    def reset(self):
        self.frame_index = 0

In [ ]:
trajectory_viewer = TrajectoryPlayback()

In [ ]:
trajectory_viewer.run_playback()

In [ ]:
trajectory_viewer.pause()

In [ ]:
# You may cancel when ready...
trajectory_viewer.cancel_playback()

Now time to clean up. We can close the Rosetta server remotely using the following command, then we will close the Narupa server.

In [ ]:
try: # For some reason the call catches despite closing the server as intended...
    client.run_command( "ROS_close_server" ) 
except:
    pass
    
narupa_server.close()
client.close()